In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


In [4]:
df_annotation.sample(10)

,width,height,slice,vcoordX,vcoordY,vcoordZ,diameterX,diameterY,diameterZ,originX,originY,originZ,spacingX,spacingY,spacingZ,label
seriesuid,,,,,,,,,,,,,,,,
651273,350,350,240,112.451172,114.843750,38.0,9.570312,10.253906,6.0,-177.100006,-27.700001,-242.820007,0.683594,0.683594,3.0,5
387453,294,294,285,186.046875,164.513672,135.0,14.355469,17.226562,15.0,-130.583603,-146.712891,-1437.117065,0.574219,0.574219,5.0,31
688423,364,364,300,212.925781,178.089844,260.0,19.906250,11.375000,5.0,-166.392334,-181.644531,-596.254150,0.710938,0.710938,5.0,5
688674,350,350,300,123.730469,211.914062,270.0,58.105469,38.964844,15.0,-171.026733,-174.658203,-71.472046,0.683594,0.683594,5.0,5
659555,296,296,300,110.132812,174.015625,285.0,5.781250,8.671875,5.0,-159.331665,-147.710938,-303.191406,0.578125,0.578125,5.0,1
524018,360,360,306,223.593750,142.382812,222.0,65.390625,77.343750,6.0,-167.699997,-180.000000,-814.299988,0.703125,0.703125,6.0,5
688514,350,350,270,186.621094,158.251953,67.0,8.886719,10.937500,10.0,-193.541885,-174.658203,-554.381470,0.683594,0.683594,5.0,31
698740,350,350,270,217.382812,173.291016,115.0,11.621094,8.203125,5.0,-174.658203,-174.658203,1231.047119,0.683594,0.683594,5.0,31
647621,376,376,265,148.343750,223.250000,245.0,27.171875,18.359375,5.0,-186.906525,-187.632812,-273.485840,0.734375,0.734375,5.0,5


# 2. Load Center Coords by Candidates

In [5]:
lungs = list(set(df_meta_test.index))
print('distinct lungs in testset:', len(lungs))

distinct lungs in testset: 222


In [6]:
list_tasks = []
size = settings.CUBE_SIZE
step = 32
start = [ size//2, size//2, size//2 ] # z,y,x

for uid in tqdm(lungs):
    meta = df_meta_test.loc[uid] # slice
    
    for z in range(start[0], int(meta.slice), step):
        for y in range(start[1], int(meta.height), step):
            for x in range(start[2], int(meta.width), step):
                task = {}
                task['seriesuid'] = uid
                task['vcoordX'] = x
                task['vcoordY'] = y
                task['vcoordZ'] = z
                
                # print(task)
                list_tasks.append(task)

100%|██████████| 222/222 [00:00<00:00, 349.63it/s]


In [7]:
df_tasks = pd.DataFrame(list_tasks, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
df_tasks = df_tasks.set_index('seriesuid')
df_tasks.index = df_tasks.index.astype('str')

print('total:', df_tasks.shape, 'lung:', len(set(df_tasks.index)))

if not os.path.exists(settings.SUBMISSION_DIR + 'candidates'):
    os.mkdir(settings.SUBMISSION_DIR + 'candidates')
    
df_tasks.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_cubes.csv', encoding='utf-8')


total: (208898, 3) lung: 222


# 3. Predict Results by tasks_cubes.csv

In [8]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

label = '31'
output_dir = './output/unet3d/1564368538/' # 31 
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

W0729 13:05:18.546121 140573377500992 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 13:05:18.547615 140573377500992 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 13:05:18.554728 140573377500992 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [10]:
batch_size = 64
threshold_probability = 0.8
wtype = 'medi'
plot = False

In [11]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [12]:
list_results = []
print(wtype, 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_test.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 

    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='test', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            print(X_test.shape)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred[:,:,:,0], itm)
                    if len(bboxes) > 0:
                        for i, box in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            if plot:
                                plot_cube((pred*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label + '.csv', encoding='utf-8')

    print('-'*100)
    

medi window
lung: 680203 tasks: 900 meta: 300.0 350.0 350.0
(900, 64, 64, 64, 1)
900/900 [==============================] - 24s 27ms/sample
total: (121, 6) lung: 1
----------------------------------------------------------------------------------------------------
lung: 688684 tasks: 900 meta: 290.0 345.0 345.0
(817, 64, 64, 64, 1)
817/817 [==============================] - 17s 21ms/sample
total: (252, 6) lung: 2
----------------------------------------------------------------------------------------------------
lung: 647539 tasks: 1089 meta: 310.0 368.0 368.0
(1058, 64, 64, 64, 1)
1058/1058 [==============================] - 23s 22ms/sample
total: (478, 6) lung: 3
----------------------------------------------------------------------------------------------------
lung: 513916 tasks: 700 meta: 255.0 332.0 332.0
(652, 64, 64, 64, 1)
652/652 [==============================] - 16s 24ms/sample
total: (636, 6) lung: 4
-------------------------------------------------------------------------

(850, 64, 64, 64, 1)
850/850 [==============================] - 15s 18ms/sample
total: (6700, 6) lung: 33
----------------------------------------------------------------------------------------------------
lung: 507932 tasks: 729 meta: 304.0 305.0 305.0
(706, 64, 64, 64, 1)
706/706 [==============================] - 12s 17ms/sample
total: (6940, 6) lung: 34
----------------------------------------------------------------------------------------------------
lung: 522077 tasks: 1452 meta: 392.0 371.0 371.0
(1241, 64, 64, 64, 1)
1241/1241 [==============================] - 26s 21ms/sample
total: (7602, 6) lung: 35
----------------------------------------------------------------------------------------------------
lung: 651598 tasks: 1100 meta: 355.0 350.0 350.0
(1021, 64, 64, 64, 1)
1021/1021 [==============================] - 21s 21ms/sample
total: (7730, 6) lung: 36
----------------------------------------------------------------------------------------------------
lung: 672957 tasks: 

(843, 64, 64, 64, 1)
843/843 [==============================] - 16s 20ms/sample
total: (13662, 6) lung: 65
----------------------------------------------------------------------------------------------------
lung: 633846 tasks: 1296 meta: 305.0 403.0 403.0
(1110, 64, 64, 64, 1)
1110/1110 [==============================] - 20s 18ms/sample
total: (13973, 6) lung: 66
----------------------------------------------------------------------------------------------------
lung: 680322 tasks: 1008 meta: 250.0 394.0 394.0
(926, 64, 64, 64, 1)
926/926 [==============================] - 17s 19ms/sample
total: (14377, 6) lung: 67
----------------------------------------------------------------------------------------------------
lung: 516840 tasks: 900 meta: 312.0 328.0 328.0
(880, 64, 64, 64, 1)
880/880 [==============================] - 18s 20ms/sample
total: (14699, 6) lung: 68
----------------------------------------------------------------------------------------------------
lung: 688883 tasks:

(907, 64, 64, 64, 1)
907/907 [==============================] - 16s 18ms/sample
total: (21627, 6) lung: 97
----------------------------------------------------------------------------------------------------
lung: 694278 tasks: 900 meta: 304.0 330.0 330.0
(749, 64, 64, 64, 1)
749/749 [==============================] - 16s 21ms/sample
total: (22038, 6) lung: 98
----------------------------------------------------------------------------------------------------
lung: 675146 tasks: 1089 meta: 300.0 371.0 371.0
(1052, 64, 64, 64, 1)
1052/1052 [==============================] - 20s 19ms/sample
total: (22503, 6) lung: 99
----------------------------------------------------------------------------------------------------
lung: 633135 tasks: 700 meta: 255.0 344.0 344.0
(636, 64, 64, 64, 1)
636/636 [==============================] - 13s 20ms/sample
total: (22890, 6) lung: 100
----------------------------------------------------------------------------------------------------
lung: 633139 tasks:

KeyboardInterrupt: 

In [13]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label +'.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))



results: (26124, 6) distinct lung: 113


In [14]:
filepath_31 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_31.csv'
filepath_32 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_32.csv'
if os.path.exists(filepath_31) and os.path.exists(filepath_32):
    df_medi_31 = pd.read_csv(filepath_or_buffer=filepath_31, index_col=['seriesuid'])
    df_medi_31.index = df_medi_31.index.astype('str')
    print('medi window 31:', df_medi_31.shape)
    
    df_medi_32 = pd.read_csv(filepath_or_buffer=filepath_32, index_col=['seriesuid'])
    df_medi_32.index = df_medi_32.index.astype('str')
    print('medi window 32:', df_medi_32.shape)
    
    df_medi = pd.concat([df_medi_31, df_medi_32])
    print('medi windows merged:', df_medi.shape)
    df_medi.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'.csv', encoding='utf-8')

medi window 31: (26124, 6)
medi window 32: (12751, 6)
medi windows merged: (38875, 6)
